In [2]:
#IMPORTING REQUIRED MODULES

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from datetime import datetime

In [3]:
#LOADED THE COVID CSV FILE AND PARSED THE DATE COLUMN AS DATE DATATYPE

newdf=pd.read_csv(r'districts.csv',parse_dates = ['Date'])

In [4]:
#REMOVED THE ROWS THAT HAVE DISTRICTNAME THAT ARE NOT DISTRICT NAME LIKE Unknown,Other State,Other Region....

df=newdf[newdf['District']!='Unknown']
df=df[df['District']!='Other State']
df=df[df['District']!='Other Region']
df=df[df['District']!='Italians']
df=df[df['District']!='Evacuees']
df=df[df['District']!='BSF Camp']
df=df[df['District']!='Airport Quarantine']
df=df[df['District']!='Railway Quarantine']
df=df[df['District']!='Foreign Evacuees']
df=df[df['District']!='Capital Complex']
df=df[df['District']!='Others']
df=df[df['District']!='State Pool']

#REMOVED UNNECESSARY COLUMNS(THAT ARE NO REQ FOR ASSIGNMENT)

del df['Recovered']
del df['Deceased']
del df['Other']
del df['Tested']

#TOOK ONLY ROWS THAT HAVE DATE BEFORE AUG 14 2021(AS IT IS THE END DATE)

df=df[df['Date']<'2021-08-15']


In [5]:
#CREATED A LIST CONTAINING ELEMETS THAT ARE STATE NAME AND STATEID


change=[['Andaman and Nicobar Islands', 'AN'],[ 'Andhra Pradesh',  'AP'],[ 'Arunachal Pradesh', 'AR'],[ 'Assam',  'AS'],[ 'Bihar',  'BR'],
 [ 'Chandigarh' ,'CH'],[ 'Chhattisgarh' , 'CT'], ['Dadra and Nagar Haveli and Daman and Diu',  'DN'],
  ['Delhi',  'DL'], ['Goa',  'GA'], ['Gujarat',  'GJ'], ['Haryana',  'HR'],[ 'Himachal Pradesh',  'HP'],
  ['Jammu and Kashmir',  'JK'],[ 'Jharkhand',  'JH'],[ 'Karnataka',  'KA'],[ 'Kerala',  'KL'],[ 'Lakshadweep', 'LD'],
 [ 'Madhya Pradesh',  'MP'],[ 'Maharashtra',  'MH'],['Manipur',  'MN'],[ 'Meghalaya','ML'],[ 'Mizoram',  'MZ'],
 ['Nagaland',  'NL'],[ 'Odisha',  'OR'],[ 'Puducherry',  'PY'],[ 'Punjab',  'PB'],['Rajasthan',  'RJ'],
 ['Sikkim',  'SK'],['Tamil Nadu', 'TN'],['Telangana',  'TG'],['Tripura','TR'],['Uttar Pradesh',  'UP'],
 ['Uttarakhand',  'UT'],['West Bengal',  'WB'],['Ladakh','LA']]


#REPLACED STATE COLUMN OF DATAFRAME WITH STATEID INPLACE OF STATE NAME USING THE ABOVE CREATED LIST

states=list(df['State'])

for i in range(len(states)):
    for j in range(len(change)):
        if states[i]==change[j][0]:
            states[i]=change[j][1]
df['State']=states

In [6]:
#CREATED A NEW COLUMN DISTRICTID BY JOINING STATEID AND DISTRICTNAME BY '_' IN THE DATAFRAME

stateid=list(df['State'])
distname=list(df['District'])

distid=[]
for i in range(len(stateid)):
    distid.append(stateid[i]+'_'+distname[i])
df['distid']=distid

In [7]:
#DELETING UNWANTED COLUMNS

del df['State']
del df['District']

In [8]:
#GROUPED THE DATAFRAME BASED ON DISTRICTID

gdf=df.groupby('distid')

In [9]:
ml=[]
for i in gdf:
    ml.append([i[0],i[1]])

In [10]:
#FOR EACH  DISTRICTID DATAFRAME ADDED NEW ROWS WITH DATES FROM 2020-03-15 TO THE MIN DATE IN THAT DATAFRAME AND WITH CASES AS 0

for i in ml:
    startdate = np.datetime64('2020-03-15')
    while(startdate < min(i[1]['Date'])):
        i[1] =  i[1].append({'Date':startdate,'distid':i[0],'Confirmed':0},ignore_index=True)
        startdate = startdate + np.timedelta64(1,'D')
    i[1] = i[1].sort_values(by=['Date'], ascending=True)

In [ ]:
#CREATED THE FINAL DATAFRAME

weekfinaldf = pd.DataFrame(columns = ['districtid','weekid','cases'])

#FOR EACH DISTRICTID DATAFRAME ,CALCULATED THE NO OF NEW  CASES IN THAT WEEK BY BELOW FORMULA AND APPEND THIS TO THE FINAL DATAFRAME
#NO OF NEW CASES IN THAT WEEK=NO OF CASES IN THE LAST DAY OF THE THAT WEEK - NO OF  CASES IN THE LAST DAY OF THE PREVIOUS  WEEK   
  
for i in ml:
    week = 1
    tempdf = pd.DataFrame(columns = ['districtid','weekid','cases'])
    startdate = np.datetime64(i[1][0:1]['Date'].to_string(index = False))
    enddate = np.datetime64(i[1][-1:]['Date'].to_string(index = False))
    datelist = list(i[1]['Date'])
    confirmedlist = list(i[1]['Confirmed'])
    while (startdate <= enddate-np.timedelta64(6,'D')):
        date2 = startdate + np.timedelta64(6,'D')
        date1 = startdate - np.timedelta64(1,'D')
        ind_date1 = 0
        ind_date2 = 0
        for j in range(0,len(datelist)):
            if datelist[j] == date1:
                ind_date1 = j
            if datelist[j] == date2:
                ind_date2 = j
        tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"weekid":week,"cases":confirmedlist[ind_date2]-confirmedlist[ind_date1]}, index=[0]))
        week = week + 1
        startdate = startdate + np.timedelta64(7,'D')
    weekfinaldf = weekfinaldf.append(tempdf)

In [ ]:
#STORED THE MONTH TO MONTH DATES IN A LIST

from datetime import datetime
mydate=list(ml[0][1]['Date'][0:1])
next_month=[]
for i in range(18):
  if(mydate[0].month+i<=12):
    next_month.append(datetime(mydate[0].year, mydate[0].month+i, 14))
  else:
    next_month.append(datetime(mydate[0].year+1, mydate[0].month+i-12, 14))


In [ ]:
#CREATED FINAL DATFRAME FOR MONTHS

monfinaldf = pd.DataFrame(columns = ['districtid','monthid','cases'])

#FOR EACH DISTRICTID DATAFRAME ,CALCULATED THE NO OF  NEW CASES IN THAT MONTH BY BELOW FORMULA AND APPEND THIS TO THE FINAL DATAFRAME
  
#NO OF NEW CASES IN THAT MONTH=NO OF CASES IN THE LAST  DAY OF THAT MONTH - NO OF CASES IN THE LAST DAY OF THE PREVIOUS MONTH
   

for i in ml:
    month = 1
    tempdf = pd.DataFrame(columns = ['districtid','monthid','cases'])
    datelist = list(i[1]['Date'])
    confirmedlist = list(i[1]['Confirmed'])
    mno=0
    while (mno<len(next_month)-1):
        date2 = next_month[mno+1]
        date1 = next_month[mno]
        ind_date1 = 0
        ind_date2 = 0
        for j in range(0,len(datelist)):
            if datelist[j] == date1:
                ind_date1 = j
            if datelist[j] == date2:
                ind_date2 = j
        tempdf = tempdf.append(pd.DataFrame({"districtid": i[0],"monthid":month,"cases":confirmedlist[ind_date2]-confirmedlist[ind_date1]}, index=[0]))
        month = month + 1
        mno+=1
    monfinaldf = monfinaldf.append(tempdf)

    

In [ ]:
#CREATED A FINALDF FOR OVERALL

overallfinaldf = pd.DataFrame(columns = ['districtid','overallid','cases'])

#FOR EACH DISTRICTID
#GOT CONFIRMED CASES FROM THE LAST DATE IN THAT DISTRICT DATAFRAME AND APPEDEDIT TO THE FINAL DATAFRAME

for i in ml:
    overall = 1
    datelist = list(i[1]['Date'])
    confirmedlist = list(i[1]['Confirmed'])
    overallfinaldf = overallfinaldf.append(pd.DataFrame({"districtid": i[0],"overallid":overall,"cases":confirmedlist[-1]}, index=[0]))
    
        

In [ ]:
#DUE TO SOME ERROR IN DATA ,THE CUMULATIVE DATA ENTRY IS NOT FOLLOWED
#SO SOME WEEKS HAVE REPORTED NEGATIVE NUMBER OF CONFIRMED CASES
#FOUND THOSE ROWS THAT HAVE NEGATIVE CONFIRMED CASES AND REPLACED THEM BY ZERO

wc=list(weekfinaldf['cases'])
for i in range(len(wc)):
    if wc[i]<0:
        wc[i]=0
mc=list(monfinaldf['cases'])
for i in range(len(mc)):
    if mc[i]<0:
        mc[i]=0
        
weekfinaldf['cases']=wc
monfinaldf['cases']=mc

In [ ]:
#FINALLY CONVERTED THE ABOVE 3 DATAFRAMES INTO cases-week.csv ,cases-month.csv,cases-overall.csv

weekfinaldf.to_csv('cases-week.csv',index=False)
monfinaldf.to_csv('cases-month.csv',index=False)
overallfinaldf.to_csv('cases-overall.csv',index=False)